<div dir="rtl" lang="pr">
    
## توضیحات پروژه

* در این پروژه موقعیت مکانی سکوهای پرتاپ و فرود راکت های پروژه اسپیس اکس و همینطور وضعیت پروژه از نظر موفقیت یا شکست پرتاب ها بر روی نقشه ترسیم می شود همچنین فاصله سکو های پرتاب از خط ساحلی محاسبه و سپس بر روی نقشه نمایش داده می شود 

<div dir="rtl" lang="pr">
    
 ## مراحل انجام پروژه
 * 1- فراخوانی کتاب خانه ها و پلاگین های مورد نیاز
 * 2- فراخوانی داده ها
 * 3- پردازش داده ها
 * 4- محاسبه فاصله بین نقاط روی نقشه
 * 5- تعریف نقشه به همراه ویژگی های آن
 * 6- تبارگذاری نقشه و پیاده سازی مختصات مورد نظر بر روی نقشه به صورت کلاستر شده و به تفکیک وضعیت پرتاب و فرود راک


</div>
 

In [1]:
# مرحله نخست

import folium
import wget
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [2]:
# مرحله دوم

spacex_df=pd.read_csv("spacex_launch_geo.csv")
spacex_df.head(2)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356


In [3]:
# مرحله سوم
#ستون های مورد نیاز را جدا می کنیم
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# بر اساس سکو داده ها را گروه بندی می کنیم.
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first() 
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [4]:
# مرحله چهارم
# تابع محاسبه فاصله دو نقطه روی نقشه
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

coastline_lat,coastline_lon=28.56321,-80.56787 # نقطه مورد نظر

launch_site_lat=launch_sites_df.loc[1][1] # مختصات سکو
launch_site_lon=launch_sites_df.loc[1][2]

# محاسبه فاصله یکی از سکو ها با نزدیک ترین خط ساحلی
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"The distance between {launch_sites_df.loc[1][0]} lonch site an coastline is about {distance_coastline}")

The distance between CCAFS SLC-40 lonch site an coastline is about 0.874348161023727


In [5]:
# مرحله پنجم
# Start location is NASA Johnson Space Center

min_lon, max_lon = -127, -67 #تعریف محدوده نقشه
min_lat, max_lat = 25, 45
nasa_coordinate = [29.559684888503615, -95.0830971930759] #نقطه آغاز نقشه 
map = folium.Map(location=nasa_coordinate,
                    max_bounds=True, #فعال کردن محدودیت نقشه
                    zoom_start=5, #میزان زوم اولیه
                    control_scale=True, #نمایش مقیاس نقشه در پایین صفحه
                    min_lat=min_lat,
                    max_lat=max_lat,
                    min_lon=min_lon,
                    max_lon=max_lon,)

# علامت گذاری محدوده مجاز
folium.CircleMarker([max_lat, min_lon], tooltip="Upper Left Corner").add_to(map)
folium.CircleMarker([min_lat, min_lon], tooltip="Lower Left Corner").add_to(map)
folium.CircleMarker([min_lat, max_lon], tooltip="Lower Right Corner").add_to(map)
folium.CircleMarker([max_lat, max_lon], tooltip="Upper Right Corner").add_to(map)

# اضافه کردن موقعیت ماوس بر روی نقشه
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

In [6]:
# مرحله ششم
# اضافه کردن دایره‌ای به شعاع دویست متر اطراف هر موقعیت
fg = folium.FeatureGroup(name="Icon collection", show=True).add_to(map) # ایجاد کلاس فیچر
for index, row in spacex_df.iterrows():# افزودن نقاط
    folium.Circle([row["Lat"],row["Long"]],radius=100, color='#d35400', fill=False).add_to(fg)
    
    folium.Marker(
    location=[row["Lat"],row["Long"]],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 18; color:#d35400;"><b>%s</b></div>' % row["Launch Site"],
        )
    ).add_to(fg)

In [7]:
# اضافه کردن مارکر وضعیت پرتاب و فعال کردن حالت کلاستر خودکار
marker_cluster = MarkerCluster().add_to(map) # ایجاد کلاستر
# اضافه کردن نقاط به نقشه
for index, row in spacex_df.iterrows():
    if  row["class"]==1:
        folium.Marker(location=[row["Lat"],row["Long"]],
                      #popup="Successful Lanch",
                      icon=folium.Icon(color="green", icon="ok-sign"),
                      ).add_to(marker_cluster)
    else:
        folium.Marker(location=[row["Lat"],row["Long"]],
                      #popup="Filed Lanch",
                      icon=folium.Icon(color="red", icon="remove-sign"),
                      ).add_to(marker_cluster)

In [8]:
# اضافه کردن فاصله از خط ساحلی و ترسیم این فاصله با یک خط مستقیم
folium.Marker(
    location=[coastline_lat,coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 18; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    ).add_to(map)

folium.PolyLine(locations=[(coastline_lat,coastline_lon),(launch_site_lat,launch_site_lon)],
    smooth_factor=50,
    color="grey",
    tooltip="Distance from coastline",
    weight=5).add_to(map)

In [9]:
# اضافه گزینه فعال و غیر فعال کردن لایه
folium.LayerControl().add_to(map) # نمایش لایه

# نمایش نقشه
map.add_child(mouse_position)